# Tables and multidimensional arrays

We are going to look to the differences between storing multidimensional data as tables and multidimensional arrays.

Tables are very flexible data structures and additional dimensions can be represented by adding additional columns to the table, at the cost of redundancy.

Multidimensional arrays excel at storing **dense** datasets while tables excel at storing **sparse** datasets.

In [1]:
import numpy as np
import pandas as pd
import h5py

# Random walker

We will work with the Random Walker dataset stored as a table, in order to contrast the differences with the array model.

In [2]:
walks = (np.random.randint(0,2,(1000,500))*2-1).cumsum(1)

In [3]:
walks

array([[  1,   0,  -1, ..., -16, -17, -18],
       [ -1,  -2,  -3, ...,  36,  37,  38],
       [  1,   0,   1, ..., -28, -29, -28],
       ...,
       [ -1,  -2,  -3, ..., -36, -35, -34],
       [ -1,   0,   1, ...,  34,  33,  32],
       [  1,   2,   1, ...,  38,  39,  38]], dtype=int32)

In [4]:
df = pd.DataFrame(walks)

In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1,0,-1,-2,-1,-2,-1,-2,-1,-2,...,-13,-12,-11,-12,-13,-14,-15,-16,-17,-18
1,-1,-2,-3,-2,-1,0,1,2,3,4,...,31,30,31,32,33,34,35,36,37,38
2,1,0,1,2,1,0,1,2,1,0,...,-29,-28,-27,-28,-27,-28,-29,-28,-29,-28
3,-1,0,1,2,1,0,-1,0,1,2,...,17,18,19,20,19,18,19,18,17,16
4,-1,0,-1,0,1,0,-1,-2,-3,-4,...,19,18,19,18,19,20,21,20,19,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,2,3,2,3,4,5,4,5,4,...,1,0,-1,-2,-1,0,1,2,1,0
996,-1,0,1,2,1,0,-1,-2,-1,-2,...,-19,-18,-17,-16,-17,-16,-17,-18,-19,-18
997,-1,-2,-3,-4,-5,-4,-5,-4,-3,-4,...,-35,-34,-35,-36,-35,-36,-37,-36,-35,-34
998,-1,0,1,0,-1,-2,-3,-4,-5,-4,...,29,30,31,32,31,32,33,34,33,32


Let's turn our matrix into a table.

In [6]:
walksdf = (df.reset_index()
             .rename({"index": "idsample"}, axis=1)
             .melt("idsample")
             .rename({"variable": "idstep", "value": "position"}, axis=1)
             .sort_values(["idsample", "idstep"]))

In [7]:
walksdf

,idsample,idstep,position
0,0,0,1
1000,0,1,0
2000,0,2,-1
3000,0,3,-2
4000,0,4,-1
...,...,...,...
495999,999,495,38
496999,999,496,37
497999,999,497,38
498999,999,498,39


## Storing tables in h5py

In order to simulate tables in HDF5, we have to use **[structured arrays](https://numpy.org/doc/stable/user/basics.rec.html)**. We can also set the dimension of our dataset to **UNLIMITED** in order to allow appending data when needed.

In [8]:
with h5py.File("p3.h5", "w") as f:
    f.create_group("walks")
    
    # store the walks matrix
    f["walks/matrix"] = walks
    
    # store the walks dataframe
    dt = np.dtype([("idsample", np.uint64), ("idstep", np.uint16), ("position", "i8")])
    shape = (len(walksdf),)
    dset = f["walks"].create_dataset(
        "dataframe",
        shape,
        dt,
        maxshape=(None,),
        chunks=(len(walksdf["idstep"].unique()),),
        compression="gzip",
        shuffle=True)

    dset[:] = np.array(
        list(zip(*walksdf.values.T)),
        dtype=dt)
    
    dset.attrs["colnames"] = ",".join(walksdf.columns)

In [9]:
!h5dump -A -p p3.h5

HDF5 "p3.h5" {
GROUP "/" {
   GROUP "walks" {
      DATASET "dataframe" {
         DATATYPE  H5T_COMPOUND {
            H5T_STD_U64LE "idsample";
            H5T_STD_U16LE "idstep";
            H5T_STD_I64LE "position";
         }
         DATASPACE  SIMPLE { ( 500000 ) / ( H5S_UNLIMITED ) }
         STORAGE_LAYOUT {
            CHUNKED ( 500 )
            SIZE 630529 (14.274:1 COMPRESSION)
         }
         FILTERS {
            PREPROCESSING SHUFFLE
            COMPRESSION DEFLATE { LEVEL 4 }
         }
         FILLVALUE {
            FILL_TIME H5D_FILL_TIME_ALLOC
            VALUE  H5D_FILL_VALUE_DEFAULT
         }
         ALLOCATION_TIME {
            H5D_ALLOC_TIME_INCR
         }
         ATTRIBUTE "colnames" {
            DATATYPE  H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            }
            DATASPACE  SCALAR
            DATA {
            (0): "idsample,i

## Working with multidimensional data as tables

Let's reproduce the exercises from Random Walker but using Pandas.

In [10]:
f = h5py.File("p3.h5")
walksdf = pd.DataFrame(f["walks/dataframe"][:],
                       columns=f["walks/dataframe"].attrs["colnames"].split(","))
walksdf = walksdf.set_index(["idsample", "idstep"])
f.close()

In [11]:
walksdf

position
idsample idstep          
0        0              1
         1              0
         2             -1
         3             -2
         4             -1
...                   ...
999      495           38
         496           37
         497           38
         498           39
         499           38

[500000 rows x 1 columns]

Print the longest position in both directions.

In [12]:
walksdf.min(), walksdf.max()

(position   -82
 dtype: int64,
 position    69
 dtype: int64)

Show the number of walks that reached either 30 steps in any of the directions.

In [13]:
hits30 = (walksdf["position"].abs() >= 30)
hits30.groupby("idsample").any().sum()

370

For the walkers that reached 30 steps in any of the directions, show the step at which they reached the longest position.

In [14]:
walksdf[hits30].reset_index().loc[walksdf[hits30].abs().reset_index().groupby("idsample")["position"].idxmax()]

,idsample,idstep,position
11,1,499,38
14,2,488,-31
16,3,369,30
47,5,205,36
469,9,483,60
...,...,...,...
34228,990,495,36
34284,993,484,39
34407,997,451,-48
34526,998,410,45


## Working with multidimensional data with xarray

xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like arrays, which allows for a more intuitive, more concise, and less error-prone developer experience. The package includes a large and growing library of domain-agnostic functions for advanced analytics and visualization with these data structures.

In [15]:
import xarray as xr

xarray contains the function `xarray.Dataset.from_dataframe()`. This function creates an `xarray.Dataset` from a `pandas.DataFrame` using the values in the index as coordinates. Let's try this function on our `walks` dataset.

In [16]:
ds = xr.Dataset.from_dataframe(walksdf)

In [17]:
ds

<xarray.Dataset>
Dimensions:   (idsample: 1000, idstep: 500)
Coordinates:
  * idsample  (idsample) uint64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * idstep    (idstep) uint64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
Data variables:
    position  (idsample, idstep) int64 1 0 -1 -2 -1 -2 -1 ... 39 38 37 38 39 38

In [18]:
ds["position"]

<xarray.DataArray 'position' (idsample: 1000, idstep: 500)>
array([[  1,   0,  -1, ..., -16, -17, -18],
       [ -1,  -2,  -3, ...,  36,  37,  38],
       [  1,   0,   1, ..., -28, -29, -28],
       ...,
       [ -1,  -2,  -3, ..., -36, -35, -34],
       [ -1,   0,   1, ...,  34,  33,  32],
       [  1,   2,   1, ...,  38,  39,  38]], dtype=int64)
Coordinates:
  * idsample  (idsample) uint64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * idstep    (idstep) uint64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499

Find the walk of the 500th walker (i.e. number 499).

In [19]:
ds["position"].sel(idsample=499) # also .isel(idsample=499) because the coordinate matches the dimension in this case

<xarray.DataArray 'position' (idstep: 500)>
array([  1,   0,  -1,   0,   1,   2,   1,   2,   1,   2,   1,   0,  -1,
        -2,  -3,  -4,  -5,  -6,  -5,  -4,  -5,  -4,  -3,  -2,  -3,  -2,
        -1,  -2,  -3,  -2,  -1,  -2,  -3,  -2,  -3,  -4,  -5,  -6,  -5,
        -4,  -5,  -4,  -3,  -4,  -5,  -4,  -5,  -6,  -7,  -8,  -9, -10,
        -9, -10,  -9,  -8,  -7,  -8,  -7,  -8,  -9,  -8,  -9,  -8,  -9,
        -8,  -7,  -8,  -7,  -8,  -9, -10, -11, -10, -11, -10,  -9, -10,
       -11, -10,  -9,  -8,  -7,  -6,  -7,  -6,  -5,  -6,  -5,  -4,  -3,
        -2,  -1,   0,   1,   0,  -1,  -2,  -1,  -2,  -3,  -4,  -3,  -4,
        -3,  -2,  -1,   0,   1,   2,   1,   0,  -1,  -2,  -3,  -2,  -3,
        -2,  -1,  -2,  -1,   0,  -1,  -2,  -1,  -2,  -1,  -2,  -3,  -4,
        -3,  -4,  -3,  -4,  -3,  -2,  -3,  -2,  -1,  -2,  -3,  -2,  -3,
        -2,  -1,  -2,  -3,  -4,  -5,  -4,  -3,  -4,  -3,  -2,  -1,  -2,
        -3,  -2,  -3,  -4,  -5,  -6,  -7,  -6,  -7,  -6,  -7,  -6,  -5,
        -6,  -5,  -6,  -5,  -6,  -7,  -8,  -9, -10,  -9, -10,  -9,  -8,
        -7,  -6,  -7,  -8,  -9, -10, -11, -12, -11, -10, -11, -12, -11,
       -10,  -9, -10,  -9, -10, -11, -10,  -9, -10,  -9, -10,  -9,  -8,
        -7,  -6,  -5,  -4,  -5,  -4,  -3,  -2,  -1,   0,  -1,  -2,  -3,
        -4,  -3,  -2,  -1,   0,   1,   0,  -1,   0,   1,   2,   1,   2,
         1,   0,  -1,   0,   1,   0,   1,   0,  -1,  -2,  -3,  -4,  -3,
        -2,  -3,  -2,  -3,  -4,  -5,  -6,  -5,  -4,  -3,  -4,  -3,  -2,
        -1,  -2,  -3,  -2,  -3,  -4,  -5,  -4,  -3,  -4,  -3,  -2,  -3,
        -2,  -3,  -2,  -3,  -4,  -5,  -6,  -5,  -6,  -7,  -6,  -5,  -6,
        -5,  -4,  -5,  -6,  -7,  -8,  -9,  -8,  -9,  -8,  -7,  -8,  -7,
        -6,  -7,  -8,  -7,  -8,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,
        -1,   0,   1,   0,  -1,  -2,  -3,  -4,  -3,  -4,  -5,  -6,  -5,
        -4,  -5,  -6,  -7,  -6,  -7,  -6,  -7,  -6,  -7,  -6,  -7,  -8,
        -9,  -8,  -7,  -8,  -7,  -6,  -5,  -6,  -7,  -6,  -7,  -8,  -7,
        -6,  -5,  -4,  -3,  -4,  -5,  -6,  -7,  -8,  -7,  -6,  -5,  -4,
        -5,  -4,  -5,  -4,  -5,  -4,  -5,  -6,  -7,  -8,  -9, -10,  -9,
       -10, -11, -12, -13, -14, -15, -14, -15, -14, -13, -12, -11, -12,
       -11, -10,  -9,  -8,  -9,  -8,  -9, -10,  -9,  -8,  -7,  -8,  -7,
        -6,  -7,  -8,  -9, -10, -11, -12, -13, -12, -13, -12, -11, -10,
        -9, -10, -11, -12, -13, -14, -15, -14, -15, -16, -17, -16, -17,
       -16, -17, -18, -17, -18, -17, -16, -15, -16, -17, -18, -17, -18,
       -17, -18, -19, -18, -19, -18, -19, -18, -17, -18, -19, -20, -21,
       -22, -23, -24, -25, -26, -27, -28, -29, -28, -29, -28, -29, -28,
       -29, -28, -29, -28, -29, -30, -29, -30, -29, -28, -27, -28, -27,
       -28, -27, -28, -29, -30, -29, -28, -29, -30, -31, -32, -31, -32,
       -33, -34, -33, -32, -33, -32], dtype=int64)
Coordinates:
    idsample  uint64 499
  * idstep    (idstep) uint64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499

Find the furthest position to the right of each walker.

In [20]:
ds["position"].max("idstep") # note that we do not use dimension number anymore but the label

<xarray.DataArray 'position' (idsample: 1000)>
array([12, 38,  2, 30, 22, 36,  9, 22,  3, 60, 36, 19, 11, 10, 14, 17, 30,
       24, 48, 15,  3, 45,  4, 33,  9, 65, 40,  5, 36,  7, 14,  3,  0, 35,
       11,  6,  8, 26,  2, 12,  1, 26, 16, 21, 43,  1, 10,  6,  8,  7,  8,
       12,  3, 20, 18, 14, 14, 31,  0,  3,  1, 25, 10,  8,  1, 29, 18,  1,
        5, 17,  3,  6, 30, 21,  4, 21,  4, 28, 19,  2,  0,  9, 69, 10, 11,
       12, 44, 20, 22, 45,  3, 16, 30,  3, 29, 16,  2,  6,  9,  7, 19,  8,
        2, 24, 13, 13,  3,  5, 19, 25, 13, 15, 39, 21,  4,  9,  2, 28, 16,
       15, 19, 20,  9,  6, 17, 47, 24,  4,  4, 28,  9,  7,  4, 27,  1,  3,
       28, 24, 63, 22, 13, 22, 12,  9, 13,  4,  5, 27,  9,  7, 14,  6,  5,
       19,  7, 23, 54, -1,  9, 26,  0, 31,  5, 19,  3, 28,  7, 34, 49, 13,
        6,  4, 47, 35, 22, 48, 28, 12, 29, 26, -1,  1, 43, 22,  2, 27, 16,
       30, 30, 27, 11, 48,  3, 32,  8,  1, 18, 29, 20,  8, 31,  4, 30,  3,
       13,  8,  1,  2, 25, 18, 14,  5, 12, 38, 42, 23, 45, 19, 34, 55, 20,
        2, 39,  5, 11,  1, -1, 27,  2, 23,  5, 23, 28, 12,  5, 49, 23, 34,
       31,  9, 14, 19, 44, 21, 21,  2,  3,  2,  8, 23, 11,  7,  4, 32, 27,
       16, 24, 11, 33,  8, 25, 10, 19, 26,  4,  5, 27, -1,  5, 62, 23, 17,
       15, 28,  3, 29, 13, 44, 24, 15,  5,  8,  1, 14,  2, 18, 17, 41,  7,
       32,  9, 16, 27, 24, 30, -1, 11, 19, 14, 32, 13,  1, 34,  7, 10,  1,
        4, 18, 11,  9, 64,  3, 27,  9, 14, 13, 24,  5, 15, 10,  6,  2,  7,
       37, 15,  1, 38, 12, 13, 11, 45,  2, 21, 12,  2, 20, 10, 36, 45, 24,
...
       13, 31,  8,  2,  4, 19, 37,  3, 20, 12, 12,  3, 11,  7, 42,  3, 34,
        1, 45,  2, 27,  5, 11,  1,  4, 39, 29, 25, 13, 11, 18, 25, 18, 29,
       29,  3,  8,  0, 19,  1,  5, 24, 31, 22, 20, 17, 21, 27,  5, 22,  5,
       25,  6, 38, 34, 34, 23, 11, 22, 12, 20,  5,  2, 45, 47,  5,  5, 23,
       23, 31, 11, 35, 35,  6, 50, 28, 17,  4,  6, 42, 10, 11, 19,  1,  7,
        9, 22, 25,  6, -1, 25, 43, 36, 58, 37, 26,  3,  2, 16,  1, 22,  3,
       10, 17, 20,  1, 11,  7, 12, 36, 20, 22, 41, 50, 40, 11,  7,  0, 36,
       27, 16,  7, 34, 34,  1, 14, 23, 41, 12, 13, 17, 12, 12, 15, 33, 10,
        3, 46,  1,  5,  2, 12, 45, 11,  6, 10, 34, 28,  5, 33,  4, 17, 13,
        9, 16, 42, -1,  3,  9, 33, 47, 12,  1,  6, 17, 36, 12,  7, 27, 36,
        1, 28, 19, 42,  4, 17,  9, 19, 50, 17, 28, 15, 31,  2, 17, 46, 38,
        9, 14,  8, 18, -1, -1, 19,  5, 19, 18,  5, 23, 18,  7,  2, 16, 14,
       19, 32, 32, 38, 25, 10, 36, 36, 20, 23,  1, 45,  4, 19, 54, 16, 42,
       12,  2, 16, 20, 10,  0, 34,  6, 15, 39, 30,  5,  4, -1, 12,  8, 47,
       11, 14, 18, 18,  9, 27, 11, 17, 15, 42,  8, 17,  5,  9, 40, 20, 22,
       21, 31, 10, 26, 38, 30, 24, 33, 25, 16, 29, 39, 24, 18, 10,  9,  4,
        9, -1, 40,  7, 12, 24,  3,  6,  5,  7, 41, 11, 20,  0, 57, 44, 10,
       42,  4,  2, 21, 19,  7,  5,  8,  9, 29,  8, 14,  2,  5,  6,  4, 33,
       11, 27, 27,  0, 36, 17, 15, 39, 11, 16,  4, -1, 45, 43],
      dtype=int64)
Coordinates:
  * idsample  (idsample) uint64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999

# VentasHechos

This time we will make an ETL process to extract data from a SQL database and store it in HDF5. We will use Pandas  to answer queries on the data stored in HDF5.

We will use the database that you have previously used in your SQL classes as a source database (available [here](https://moodle.unican.es/mod/resource/view.php?id=734274)). You will answer the same questions you have already answered using SQL (available [here](https://moodle.unican.es/pluginfile.php/1109178/mod_resource/content/2/Ejercicio%2011%20-%20sqlOLAP.pdf)).

In [21]:
import sqlite3

DBFILE = 'olapMaster.db'

In [22]:
conn = sqlite3.connect('olapMaster.db')
c = conn.cursor()

c.execute('select * from ventasHechos')
ventasHechos = c.fetchall() # problem if the table contains millions of rows

for venta in ventasHechos:
    print(venta)

c.close()
conn.close()

(1, 20170202, 1, 1, 1, 2.0, 19.95, 8.35, 11.6)
(2, 20170202, 4, 2, 1, 2.0, 19.75, 7.45, 12.3)
(3, 20170202, 1, 3, 1, 1.0, 9.9, 4.95, 4.95)
(4, 20170507, 1, 1, 1, 1.0, 7.95, 4.95, 3)
(5, 20160823, 4, 5, 1, 7.0, 42.55, 28.95, 13.599999999999998)
(6, 20160202, 1, 1, 1, 2.0, 18.95, 9.95, 9)
(7, 20161010, 1, 4, 1, 4.0, 20.59, 16.95, 3.6400000000000006)
(8, 20160812, 5, 1, 7, 2.0, 12.95, 10.95, 2)
(9, 20150412, 5, 2, 7, 5.0, 25.95, 20.95, 5)
(10, 20150810, 4, 4, 6, 7.0, 6.99, 4.49, 2.5)
(11, 20150809, 3, 3, 6, 3.0, 3.99, 1.49, 2.5)
(12, 20150710, 5, 3, 5, 2.0, 70.1, 30.1, 39.99999999999999)
(13, 20170404, 2, 1, 3, 1.0, 56.09, 30.09, 26.000000000000004)
(14, 20150303, 2, 1, 3, 1.0, 53.09, 30.09, 23.000000000000004)
(16, 20150302, 2, 5, 4, 1.0, 78.15, 40.35, 37.800000000000004)
(17, 20161010, 4, 2, 3, 3.0, 123.99, 90.09, 33.89999999999999)


In [23]:
conn = sqlite3.connect('olapMaster.db')

ventasHechos = pd.read_sql_query('select * from ventasHechos', conn)

for venta in ventasHechos:
    print(venta)


conn.close()
dt = ventasHechos.dtypes

idventa
idfecha
idvendedor
idusuario
idproducto
cantidad
precio
coste
beneficio


## SQL to HDF5

Append to the HDF5 file a group named `sales`. The contents of the group will be the same tables found in the SQL database.

In [24]:
# extract data from SQL database

In [25]:
# Incluimos en variables el contenido de las tablas
conn = sqlite3.connect('olapMaster.db')
c = conn.cursor()

c.execute('select * from ventasHechos')
ventasHechos = c.fetchall()

c.execute('select * from fechaDim')
fechaDim = c.fetchall()

c.execute('select * from productoDim')
productoDim = c.fetchall()

c.execute('select * from usuarioDim')
usuarioDim = c.fetchall()

c.execute('select * from vendedorDim')
vendedorDim = c.fetchall()

c.close()
conn.close()


In [26]:
with h5py.File("p3.h5", "r+") as f:
    f.create_group("sales")
    
    ### ventasHechos ###
    dt = [('idventa', 'i8'), ('idfecha', 'i8'), ('idvendedor', 'i8'), ('idusuario', 'i8'),
          ('idproducto', 'i8'), ('cantidad', 'f8'), ('precio', 'f8'), ('coste', 'f8'), ('beneficio', 'f8')]
    shape = len(ventasHechos)
    dset = f["sales"].create_dataset(
        "ventasHechos",
        shape,
        dt,
        maxshape=(None,))
    
    dset[:] = ventasHechos
    
    ### fechaDim ###
    dt = [('idfecha', 'i8'), ('fecha', 'i8'),
          ('diaMes', h5py.string_dtype(encoding='utf-8')),
          ('diaNombre', h5py.string_dtype(encoding='utf-8')),
          ('diaAnio', h5py.string_dtype(encoding='utf-8')),
          ('semanaMes', h5py.string_dtype(encoding='utf-8')),
          ('semanaAnio', h5py.string_dtype(encoding='utf-8')),
          ('mes', h5py.string_dtype(encoding='utf-8')),
          ('nombreMes', h5py.string_dtype(encoding='utf-8')),
          ('anio', h5py.string_dtype(encoding='utf-8')),
          ('esFinDeSemana', 'i8'), ('semestre', 'i8'), ('trimestre', 'i8')]
    shape = len(fechaDim)
    dset = f["sales"].create_dataset(
        "fechaDim",
        shape,
        dt,
        maxshape=(None,))
    
    # Tratamos la aparicion de None en las columnas esFinDeSemana y semestre
    for i, fecha in enumerate(fechaDim):
        fecha = list(fecha)    #Necesario transformar a lista porque las tuplas son inmutables
        for j, e in enumerate(fecha):
            if e == None:
                fecha[j] = 0
        fecha = tuple(fecha)
        fechaDim[i] = fecha
        
    dset[:] = fechaDim
    
    ### productoDim ###
    dt = [('idproducto', 'i8'),
          ('nombreProducto', h5py.string_dtype(encoding='utf-8')),
          ('precioVentaActual', 'f8'), ('costeActual', 'f8'),
          ('categoria', h5py.string_dtype(encoding='utf-8'))]
    shape = len(productoDim)
    dset = f["sales"].create_dataset(
        "productoDim",
        shape,
        dt,
        maxshape=(None,)) 
    
    dset[:] = productoDim
    
    ### usuarioDim ###
    dt = [('idusuario', 'i8'),
          ('nombreYapellidos', h5py.string_dtype(encoding='utf-8')),
          ('genero', h5py.string_dtype(encoding='utf-8')),
          ('regionUsuario', h5py.string_dtype(encoding='utf-8'))]
    shape = len(usuarioDim)
    dset = f["sales"].create_dataset(
        "usuarioDim",
        shape,
        dt,
        maxshape=(None,)) 
    
    dset[:] = usuarioDim
    
    ### vendedorDim ###
    dt = [('idvendedor', 'i8'),
          ('nombreVendedor', h5py.string_dtype(encoding='utf-8')),
          ('regionVendedor', h5py.string_dtype(encoding='utf-8'))]
    shape = len(vendedorDim)
    dset = f["sales"].create_dataset(
        "vendedorDim",
        shape,
        dt,
        maxshape=(None,)) 
    
    dset[:] = vendedorDim

In [27]:
!h5dump -A p3.h5

HDF5 "p3.h5" {
GROUP "/" {
   GROUP "sales" {
      DATASET "fechaDim" {
         DATATYPE  H5T_COMPOUND {
            H5T_STD_I64LE "idfecha";
            H5T_STD_I64LE "fecha";
            H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            } "diaMes";
            H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            } "diaNombre";
            H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            } "diaAnio";
            H5T_STRING {
               STRSIZE H5T_VARIABLE;
               STRPAD H5T_STR_NULLTERM;
               CSET H5T_CSET_UTF8;
               CTYPE H5T_C_S1;
            } "semanaMes";
            H5T_STRING {
               STRSIZ

## Analysis with Pandas

Answer the questions from the PDF file using Pandas.

In [28]:
f = h5py.File("p3.h5")
g = f["sales"]

Query the data from every dimension table. Study the meaning of each field.

In [29]:
fechaDimdf = pd.DataFrame(f["sales/fechaDim"][:])
fechaDimdf

,idfecha,fecha,diaMes,diaNombre,diaAnio,semanaMes,semanaAnio,mes,nombreMes,anio,esFinDeSemana,semestre,trimestre
0,20140101,2014,b'1',b'Mi\xc3\xa9rcoles',b'1',b'1',b'1',b'1',b'Enero',b'2014',0,1,1
1,20140102,2014,b'2',b'Jueves',b'2',b'1',b'1',b'1',b'Enero',b'2014',0,1,1
2,20140103,2014,b'3',b'Viernes',b'3',b'1',b'1',b'1',b'Enero',b'2014',0,1,1
3,20140104,2014,b'4',b'S\xc3\xa1bado',b'4',b'1',b'1',b'1',b'Enero',b'2014',1,1,1
4,20140105,2014,b'5',b'Domingo',b'5',b'1',b'1',b'1',b'Enero',b'2014',1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324,20171224,2017,b'24',b'Domingo',b'358',b'*',b'52',b'12',b'Diciembre',b'2017',1,2,4
1325,20171225,2017,b'25',b'Lunes',b'359',b'*',b'53',b'12',b'Diciembre',b'2017',0,2,4
1326,20171226,2017,b'26',b'Martes',b'360',b'*',b'53',b'12',b'Diciembre',b'2017',0,2,4
1327,20171227,2017,b'27',b'Mi\xc3\xa9rcoles',b'361',b'*',b'53',b'12',b'Diciembre',b'2017',0,2,4


In [30]:
productoDimdf = pd.DataFrame(f["sales/productoDim"][:])
productoDimdf

,idproducto,nombreProducto,precioVentaActual,costeActual,categoria
0,1,b'PowerBank USB 5500 mAh',9.95,4.95,b'electronica'
1,2,b'Bater\xc3\xada HP dv7 8000 mAh',13.49,6.49,b'electronica'
2,3,b'C\xc3\xa1mara fotogr\xc3\xa1fica 13.4 mpx',104.35,40.35,b'electronica'
3,4,b'Mesa plegable 200x170m',56.09,30.09,b'hogar'
4,5,b'Silla de escritorio',35.10,10.10,b'hogar'
5,6,b'Patatas fritas sabor anchoa 200gr',1.49,0.49,b'alimentacion'
6,7,b'Sobaos del Pas 1kg',7.95,5.95,b'alimentacion'
7,8,b'Corbatas de Unquera 500 grs',5.75,3.75,b'alimentacion'


In [31]:
usuarioDimdf = pd.DataFrame(f["sales/usuarioDim"][:])
usuarioDimdf

,idusuario,nombreYapellidos,genero,regionUsuario
0,1,b'Laro Bustamante',b'M',b'Cantabria'
1,2,b'Jana Coss\xc3\xado',b'F',b'Cantabria'
2,3,b'Deva Cobo',b'F',b'Cantabria'
3,4,b'Teresa Ruiz',b'F',b'Rioja'
4,5,b'Guillermo Roncesvalles',b'M',b'Rioja'
5,6,b'Mariano Zapatero',b'M',b'Asturias'


In [32]:
vendedorDimdf = pd.DataFrame(f["sales/vendedorDim"][:])
vendedorDimdf

,idvendedor,nombreVendedor,regionVendedor
0,1,b'Electr\xc3\xb3nica Li\xc3\xa9bana',b'Cantabria'
1,2,b'Muebles Sotileza',b'Cantabria'
2,3,b'Alimentaci\xc3\xb3n Riojana',b'Rioja'
3,4,b'DeTodoUnPoco Pelayo',b'Cantabria'
4,5,b'Cahopos y m\xc3\xa1s',b'Asturias'


Perform the same task for the dataset `ventasHechos`.

In [33]:
ventasHechosdf = pd.DataFrame(f["sales/ventasHechos"][:])
ventasHechosdf

,idventa,idfecha,idvendedor,idusuario,idproducto,cantidad,precio,coste,beneficio
0,1,20170202,1,1,1,2.0,19.95,8.35,11.60
1,2,20170202,4,2,1,2.0,19.75,7.45,12.30
2,3,20170202,1,3,1,1.0,9.90,4.95,4.95
3,4,20170507,1,1,1,1.0,7.95,4.95,3.00
4,5,20160823,4,5,1,7.0,42.55,28.95,13.60
5,6,20160202,1,1,1,2.0,18.95,9.95,9.00
6,7,20161010,1,4,1,4.0,20.59,16.95,3.64
7,8,20160812,5,1,7,2.0,12.95,10.95,2.00
8,9,20150412,5,2,7,5.0,25.95,20.95,5.00
9,10,20150810,4,4,6,7.0,6.99,4.49,2.50


In [34]:
f.close()

Get the sales of 2017 (**Hint**: `pd.merge()`)

In [35]:
df = fechaDimdf.merge(ventasHechosdf, on = "idfecha")
df[df.fecha == 2017][ventasHechosdf.keys()]

,idventa,idfecha,idvendedor,idusuario,idproducto,cantidad,precio,coste,beneficio
11,1,20170202,1,1,1,2.0,19.95,8.35,11.60
12,2,20170202,4,2,1,2.0,19.75,7.45,12.30
13,3,20170202,1,3,1,1.0,9.90,4.95,4.95
14,13,20170404,2,1,3,1.0,56.09,30.09,26.00
15,4,20170507,1,1,1,1.0,7.95,4.95,3.00


Get the sales of the first six months of the year.

In [62]:
#Transformamos la columna a tipo int
df.mes = df.mes.astype(str).astype(int)
df[df.mes<7][ventasHechosdf.keys()]

,idventa,idfecha,idvendedor,idusuario,idproducto,cantidad,precio,coste,beneficio
0,16,20150302,2,5,4,1.0,78.15,40.35,37.80
1,14,20150303,2,1,3,1.0,53.09,30.09,23.00
2,9,20150412,5,2,7,5.0,25.95,20.95,5.00
6,6,20160202,1,1,1,2.0,18.95,9.95,9.00
11,1,20170202,1,1,1,2.0,19.95,8.35,11.60
12,2,20170202,4,2,1,2.0,19.75,7.45,12.30
13,3,20170202,1,3,1,1.0,9.90,4.95,4.95
14,13,20170404,2,1,3,1.0,56.09,30.09,26.00
15,4,20170507,1,1,1,1.0,7.95,4.95,3.00


Get the total, average, max and min benefit from all the sales.

```sql
select sum(beneficio), avg(beneficio), max(beneficio), min(beneficio) from ventasHechos;
```

In [64]:
print(ventasHechosdf.beneficio.sum(), "\t",
      ventasHechosdf.beneficio.mean(), "\t",
      ventasHechosdf.beneficio.max(), "\t",
      ventasHechosdf.beneficio.min())

230.79 	 14.424375 	 39.99999999999999 	 2.0


Repeat the last exercise grouping by year.

```sql
select anio, sum(beneficio), avg(beneficio), max(beneficio), min(beneficio) 
from ventasHechos vh inner join fechaDim fd on vh.idfecha = fd.idfecha
group by anio;
```

In [65]:
df.groupby("fecha").agg({"beneficio": [np.sum, np.mean, np.max, np.min]})

beneficio                      
            sum       mean  amax amin
fecha                                
2015     110.80  18.466667  40.0  2.5
2016      62.14  12.428000  33.9  2.0
2017      57.85  11.570000  26.0  3.0

## Analysis with xarray

Execute the following code to create a Pandas DataFrame that holds the data variables (cantidad, precio, coste, beneficio) in the columns and the coordinates/dimensions in the index (idfecha, idusuario, idproducto, idvendedor).

Then use the function `xarray.Dataset.from_dataframe` to create an `xarray.Dataset` object and examine it.

Perform the following queries of your choice using the `xarray.Dataset` object. Refer to the [tutorial](http://xarray.pydata.org/en/stable/getting-started-guide/quick-overview.html) if you need guiadance:

- Get the benefits (variable "beneficio") of sales for user "1".
- Get the average quantity (variable "cantidad") sold from seller "2" along the "idusuario" dimension.
- Note that the variables (multidimensional arrays) are full of NaNs, what is the cause of this?

In [66]:
with h5py.File("p3.h5") as f:
    df = (pd.DataFrame(f["sales/ventasHechos"][:])
          .drop("idventa", axis=1)
          .set_index(["idfecha", "idproducto", "idusuario", "idvendedor"]))

In [67]:
ventasxr = xr.Dataset.from_dataframe(df)

In [68]:
#Get the benefits (variable "beneficio") of sales for user "1"
ventasxr["beneficio"].sel(idusuario = 1)

<xarray.DataArray 'beneficio' (idfecha: 13, idproducto: 6, idvendedor: 5)>
array([[[ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan],
        [ nan, 23. ,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]],
...
       [[11.6,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan],
        [ nan, 26. ,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]],

       [[ 3. ,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan]]])
Coordinates:
  * idfecha     (idfecha) int64 20150302 20150303 20150412 ... 20170404 20170507
  * idproducto  (idproducto) int64 1 3 4 5 6 7
    idusuario   int64 1
  * idvendedor  (idvendedor) int64 1 2 3 4 5

In [69]:
#Get the average quantity (variable "cantidad") sold from seller "2" along the "idusuario" dimension.
ventasxr["cantidad"].sel(idvendedor = 2).mean("idusuario")

<xarray.DataArray 'cantidad' (idfecha: 13, idproducto: 6)>
array([[nan, nan,  1., nan, nan, nan],
       [nan,  1., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan,  1., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]])
Coordinates:
  * idfecha     (idfecha) int64 20150302 20150303 20150412 ... 20170404 20170507
  * idproducto  (idproducto) int64 1 3 4 5 6 7
    idvendedor  int64 2

In [70]:
#Note that the variables (multidimensional arrays) are full of NaNs, what is the cause of this?

Esto se debe a que estamos introduciendo los datos de una tabla (ideal para trabajar con datasets **limitados**) en un array multidimensional (al que se le saca más partido con datasets **densos**), por lo que ser introducen los valores de la tabla `ventasHechos`, donde tenemos 16 filas y 9 columnas, pero el resto de elementos del array quedan vacíos. Notar que se trata de 13x6x5x5 elementos, que obviamente no podemos rellenar con los 16x9 elementos de la tabla `ventasHechos`.